In [ ]:
#| default_exp ai

# 🧠 AI-Powered Text Editing

This module provides structured text editing using OpenAI's API with Pydantic validation.

**Features:**
- Structured outputs via Pydantic models ensuring type-safe operations
- Iterative editing with full context preservation
- Efficient conversation management (single assistant message + cumulative user instructions)
- Natural language support for commands like "change it back" or "undo that"

In [ ]:
#| export
from typing import List, Dict, Literal, Union
from pydantic import BaseModel, ConfigDict, Field, model_validator
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

# Load API key from .env file
load_dotenv()
_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 📦 Schema Models

Pydantic models ensure the AI returns valid, structured JSON:

**Container:**
- `EditPlan` — holds a list of operations to execute sequentially

**Operations:**
- `ReplaceAllOp` — replace all occurrences of exact text
- `RegexReplaceOp` — pattern-based replacement with capture groups
- `InsertAtOp` — insert at absolute character position (0-indexed)
- `InsertAfterOp` — insert after first occurrence of marker text
- `DeleteOp` — delete first or all occurrences of text

These schemas act as a contract with validation errors instead of silent failures.

In [ ]:
#| export

# --- Replace all ------------------------------------------------------------

class ReplaceAllOp(BaseModel):
    """Represents a 'replace all' text operation."""
    op: Literal["replace_all"]
    find: str = Field(..., min_length=1)
    replace: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")


# --- Regex replace ------------------------------------------------------------

class RegexReplaceOp(BaseModel):
    """Represents a regex-based find/replace operation."""
    op: Literal["regex_replace"]
    pattern: str = Field(..., min_length=1)
    replacement: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")

    @model_validator(mode="after")
    def _validate_regex(cls, v: "RegexReplaceOp"):
        # Precompile regex to ensure it's valid
        try:
            re.compile(v.pattern)
        except re.error as e:
            raise ValueError(f"Invalid regex pattern: {e}") from e
        return v

# --- Insert at absolute position ---------------------------------------------

class InsertAtOp(BaseModel):
    """Insert text at an absolute character position (0-indexed)."""
    op: Literal["insert_at"]
    text: str = Field(..., min_length=1)
    position: int = Field(..., ge=0)
    model_config = ConfigDict(extra="forbid")

# --- Insert after marker ------------------------------------------------------

class InsertAfterOp(BaseModel):
    """Insert text after the first occurrence of a marker string."""
    op: Literal["insert_after"]
    text: str = Field(..., min_length=1)
    after: str = Field(..., min_length=1)
    model_config = ConfigDict(extra="forbid")

# --- Delete -------------------------------------------------------------------

class DeleteOp(BaseModel):
    """Delete exact text (first or all occurrences)."""
    op: Literal["delete"]
    text: str = Field(..., min_length=1)
    all_occurrences: bool = False
    model_config = ConfigDict(extra="forbid")


# --- Edit plan container ------------------------------------------------------

class EditPlan(BaseModel):
    """Represents a list of text operations to apply sequentially."""
    ops: List[
        Union[
            ReplaceAllOp,
            RegexReplaceOp,
            InsertAtOp,
            InsertAfterOp,
            DeleteOp,
        ]
    ]
    model_config = ConfigDict(extra="forbid")

## 🧰 Conversation Management

The AI conversation uses a hybrid context pattern for efficiency.

**Session State:**
- `_messages` — conversation history
- `_current` — current transcript after applied edits

**Structure:**
- **System message:** defines AI role and available operations
- **Assistant message:** contains current transcript (updated after each edit)
- **User messages:** cumulative instruction history

**Example after 2 edits:**
```json
[
  {"role": "system", "content": "You are a precise text editor..."},
  {"role": "assistant", "content": "Here is the current transcript:\nI met oscar on Monday."},
  {"role": "user", "content": "Instruction: Change him to oscar"},
  {"role": "user", "content": "Instruction: Change yesterday to on Monday"}
]
```

**Functions:**
- `_new_conversation(transcript)` — initializes conversation with system and assistant messages
- `_set_current_transcript(new_transcript)` — updates assistant message with latest transcript

In [ ]:
# | export
# --- session state (module-level) ---
_messages: List[Dict[str, str]] | None = None
_current: str | None = None

def _new_conversation(transcript: str) -> List[Dict[str, str]]:
    """Create a new message list with system + assistant context."""
    return [
        {
            "role": "system",
            "content": (
                "You are a precise text editor that outputs ONLY valid JSON matching the EditPlan schema.\n\n"
                "Available operations:\n"
                "1. replace_all — exact literal text only (no regex)\n"
                "   fields:\n"
                "       - find: the exact text to replace\n"
                "       - replace: replacement text for every occurrence\n\n"
                "2. regex_replace - pattern-based replacements (e.g., dates)\n"
                "   fields:\n"
                "       - pattern: regex pattern to match (e.g., (\\d{4})-(\\d{2})-(\\d{2}) for dates)\n"
                "       - replacement: replacement string using \\1, \\2 for capture groups\n\n"
                "3. insert_at — insert text at an absolute index (0 = start)\n"
                "   fields:\n"
                "       - text: text to insert\n"
                "       - position: integer index to insert at\n\n"
                "4. insert_after — insert text after a marker\n"
                "   fields:\n"
                "       - text: text to insert\n"
                "       - after: insert after the first occurrence of this string\n"
                "       (ALWAYS provide a space in the string if needed when doing insert)\n\n"
                "5. delete — remove exact text\n"
                "   fields:\n"
                "       - text: the exact text to remove\n"
                "       - all_occurrences: true = remove all, false = only first (default false)\n\n"
                "RULES:\n"
                "- If you see regex patterns or date formats, you MUST use regex_replace, NOT replace_all!\n"
                "- When interpreting natural or spoken language, infer the user's intent precisely and map it to the correct fields.\n"
                "- ALWAYS provide a space in text to insert if needed.\n"
                "- Respond ONLY with valid JSON following the EditPlan schema."
            ),
        },
        {
            "role": "assistant",
            "content": f"Current text to edit:\n{transcript}",
        },
    ]


def _set_current_transcript(new_transcript: str) -> None:
    global _messages
    # replace the single assistant transcript message
    for m in _messages:
        if m.get("role") == "assistant":
            m["content"] = f"Current text to edit:\n{new_transcript}"
            return
    # Fallback: insert one if missing
    _messages.insert(1, {
        "role": "assistant",
        "content": f"Current text to edit:\n{new_transcript}",
    })

## 🤖 Core Functions

**`_plan_edits(instruction, model)`**
- Appends user instruction to conversation
- Calls OpenAI API with `response_format=EditPlan` for structured output
- Returns parsed `EditPlan` object

**`_apply_plan(transcript, plan)`**
- Applies all operations in `EditPlan` sequentially to the transcript
- Supports: `replace_all`, `regex_replace`, `insert_at`, `insert_after`, `delete`
- Returns updated transcript

In [ ]:
# | export
import re

def _plan_edits(instruction: str, model: str = "gpt-4o-mini") -> EditPlan:
    """
    Append a user instruction, call the model with structured output, and return the parsed plan.
    """
    global _messages, _client
    # Add the new instruction to the conversation
    _messages.append({"role": "user", "content": f"Instruction: {instruction}"})

    completion = _client.chat.completions.parse(
        model=model,
        messages=_messages,
        response_format=EditPlan,  # enforce strict structured output
        temperature=0
    )
    msg = completion.choices[0].message
    if msg.refusal:
        raise RuntimeError(f"Model refused: {msg.refusal}")
    return msg.parsed


def _apply_plan(transcript: str, plan: EditPlan) -> str:
    """
    Apply all operations from the EditPlan to the given transcript.
    """
    updated = transcript
    for op in plan.ops:
        if op.op == "replace_all":
            updated = updated.replace(op.find, op.replace)
        elif op.op == "regex_replace":
            updated = re.sub(op.pattern, op.replacement, updated)
        elif op.op == "insert_at":
            pos = max(0, min(op.position, len(updated)))
            updated = updated[:pos] + op.text + updated[pos:]
        elif op.op == "insert_after":
            idx = updated.find(op.after)
            if idx != -1:
                insert_pos = idx + len(op.after)
                updated = updated[:insert_pos] + op.text + updated[insert_pos:]
        elif op.op == "delete":
            if op.all_occurrences:
                updated = updated.replace(op.text, "")
            else:
                # Delete first occurrence only
                idx = updated.find(op.text)
                if idx != -1:
                    updated = updated[:idx] + updated[idx + len(op.text):]
    return updated

## 🔌 Public API

Functions for managing edit sessions and applying instructions.

In [ ]:
# | export
def has_session() -> bool:
    """Return True if an edit session is initialized."""
    return _messages is not None and _current is not None

def start_session(initial_transcript: str) -> str:
    """Seed a new session with the initial transcript and return it."""
    global _messages, _current
    _current = initial_transcript
    _messages = _new_conversation(initial_transcript)
    return _current

def apply_instruction(instruction: str) -> str:
    """Apply an instruction to the current transcript and return the updated text."""
    global _current
    if not has_session():
        raise RuntimeError("No session. Call start_session() first.")
    plan = _plan_edits(instruction)
    _current = _apply_plan(_current, plan)
    _set_current_transcript(_current)
    return _current

def current_transcript() -> str:
    """Get the latest edited transcript (or '' if none)."""
    return _current or ""

def reset_session() -> None:
    """Clear session state."""
    global _messages, _current
    _messages, _current = None, None

## 🧪 Example: Sequential Editing

Demonstrates two editing steps:
1. Replace all "him" with "oscar"
2. Replace "yesterday" with "on Monday"

After each step, the plan is applied and the assistant's transcript is updated while preserving all user instructions for context.

In [ ]:
#| eval: false
# Initial transcript
current_transcript = "I told him that I saw him yesterday. Then I asked him if he could help."
_messages = _new_conversation(current_transcript)

# 1️⃣ First instruction
plan1 = _plan_edits("Change all occurrences of 'him' to 'oscar'.")
current_transcript = _apply_plan(current_transcript, plan1)
_set_current_transcript(current_transcript)

# 2️⃣ Next instruction
plan2 = _plan_edits("Now change 'yesterday' to 'on Monday'.")
current_transcript = _apply_plan(current_transcript, plan2)
_set_current_transcript(current_transcript)

print("✅ Final updated transcript:\n", current_transcript)


## 🧩 Inspecting the Conversation

Print the `_messages` list to see what the model sees on each call.

**Key observations:**
- One assistant message with the current transcript
- Multiple user instructions recording session history

In [ ]:
#| eval: false
from pprint import pprint

print("🧩 Message history:")
pprint(_messages)


## ✅ Summary

**Architecture:**
- Hybrid context: single assistant message (current state) + cumulative user instructions (history)
- Efficient for long transcripts with complex edit sequences
- Supports natural, conversational editing patterns

**Supported Operations:**
1. `replace_all` — exact text replacement
2. `regex_replace` — pattern-based with capture groups (\1, \2, etc.)
3. `insert_at` — insert at character position (0-indexed)
4. `insert_after` — insert after marker string
5. `delete` — remove first or all occurrences

**Future Enhancements:**
- Token usage tracking
- Operation history with undo/redo
- UI integration (TUI/web)

## 🧪 Testing New Operation Types

Let's test all the new operation types with comprehensive examples.

In [ ]:
#| eval: false
### Test 1: Regex Replace - Format dates

initial_transcript = "Meeting on 2025-10-07 and another on 2025-12-25."
_messages = _new_conversation(initial_transcript)
_current = initial_transcript

# Use regex to convert dates from YYYY-MM-DD to MM/DD/YYYY
plan = _plan_edits("Convert all dates from YYYY-MM-DD format to MM/DD/YYYY format")
_current = _apply_plan(_current, plan)
print(f"Original: {initial_transcript}")
print(f"Result:   {_current}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#| eval: false
### Test 2: Insert_at position

test_text = "HelloWorld"
_messages = _new_conversation(test_text)
_current = test_text

plan = _plan_edits("Hello world needs a space")
result = _apply_plan(_current, plan)
print(f"Original: {test_text}")
print(f"Result:   {result}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#| eval: false
### Test 3: Insert_after - Add text after marker

test_text = "Hello, my name is John. I love coding."
_messages = _new_conversation(test_text)
_current = test_text

plan = _plan_edits("Add Smith after John")
result = _apply_plan(_current, plan)
print(f"Original: {test_text}")
print(f"Result:   {result}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#| eval: false
### Test 4: Delete - Single occurrence

test_text = "I like apples and apples are great!"
_messages = _new_conversation(test_text)
_current = test_text

plan = _plan_edits("Delete the first occurrence of 'apples'")
result = _apply_plan(_current, plan)
print(f"Original: {test_text}")
print(f"Result:   {result}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#| eval: false
### Test 5: Delete - All occurrences

test_text = "I like apples and apples are great!"
_messages = _new_conversation(test_text)
_current = test_text

plan = _plan_edits("Delete all occurrences of 'apples'")
result = _apply_plan(_current, plan)
print(f"Original: {test_text}")
print(f"Result:   {result}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#| eval: false
### Test 6: Complex multi-operation edit

test_text = "The meeting is on 2025-10-07 at the office. Please confirm."
_messages = _new_conversation(test_text)
_current = test_text

plan = _plan_edits("Change the date format to MM/DD/YYYY, change 'office' to 'conference room', and add '(urgent)' at the end")
result = _apply_plan(_current, plan)
print(f"Original: {test_text}")
print(f"Result:   {result}")
print(f"Plan: {plan.model_dump_json(indent=2)}")

In [ ]:
#from smolagents import CodeAgent, InferenceClientModel, WebSearchTool
#https://huggingface.co/docs/smolagents/index


# Connect to running vLLM server using OpenAI-compatible API
# model = OpenAIServerModel(
#     model_id="Qwen/Qwen3-4B-Instruct-2507",
#     api_base="http://localhost:8000/v1",
#     api_key="dummy",  # vLLM doesn't require a real API key
#     temperature=0.1,  # Lower temperature for more consistent output
# )

# model = InferenceClientModel()
# agent = CodeAgent(
#     tools=[WebSearchTool()],
#     model = model 
# )

# # Test with a simple question first
# print("Testing simple question...")
# response = agent.run("What is the square root of 75?")
# print(f"Response: {response}")